In [1]:
import MySQLdb as mdb
import os
import json
from stop_words import get_stop_words
from nltk import collocations

In [3]:
BASE_DIR = os.path.abspath('..')

with open(os.path.join(BASE_DIR, '.secure.settings.json')) as secret:
    SECRET = json.loads(secret.read())

SECRET_KEY = SECRET["SECRET_KEY"]

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'TEST_CHARSET': 'UTF8',
        'HOST': '',
        'PORT': '3306',
    }
}

DATABASES['default']['NAME'] = SECRET['DEV_DATABASES_NAME']
DATABASES['default']['USER'] = SECRET['DEV_DATABASES_USER']
DATABASES['default']['PASSWORD'] = SECRET['DEV_DATABASES_PASSWORD']

USER = DATABASES['default']['USER']
PASSWORD = DATABASES['default']['PASSWORD']
NAME = DATABASES['default']['NAME']


class Database(object):
    """Класс для общения с базой данных MySQL"""

    def __init__(self):
        """Создать соединение с базой данных.

        В параметрах соединения указывается хост, логин, пароль, название базы данных, кодировка.
        """
        self._connection = mdb.connect('', USER, PASSWORD, NAME, charset='utf8')

    def commit(self):
        self._connection.commit()

    def execute(self, q):
        """Вернуть результат выполнения запроса в виде массива кортежей.

        Каждый кортеж - строка базы данных, сформированная по запросу.
        """
        self.cur = self._connection.cursor()  # mdb.cursors.DictCursor
        self.cur.execute(q)
        res = self.cur.fetchall()
        self.cur.close()
        return res

In [12]:
# количество студентов
req1 = 'SELECT author FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="FL"'

db = Database()
students = set()
for i in db.execute(req1):
        students.add(i[0])

print(len(students))

38


In [14]:
# количество студентов
req1 = 'SELECT author FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="HL"'

db = Database()
students = set()
for i in db.execute(req1):
        students.add(i[0])

print(len(students))

40


In [5]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC"'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

texts = []
for n in rulec:
    f = open('texts/' + str(n) + '.txt', 'w', encoding='utf-8')
    req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
    text = ''
    for i in db.execute(req2):
        text += i[0]
    texts.append(text)
    f.write(text)
    f.close()

In [6]:
len(texts)

2905

In [17]:
stop_words = get_stop_words('ru')
for word in ['свой', '&lt;', '«', '–', '-', '(', ')', '»', 'lt', '', 
             'из-за', 'gt', 'какой-то', 'д', 'name', 'ого', 'ом']:
    stop_words.append(word)

In [3]:
with open('stop_words.txt', 'w', encoding='utf-8') as fw:
    for word in stop_words:
        fw.write(word + '\n')

In [11]:
def freq(texts):
    dic = {}

    for text in texts:
        words = text.split()
        for indx, word in enumerate(words):
            word = word.lower()
            words[indx] = word.strip('.,!@?#$%^&*():"№;%=-,')
            if words[indx] not in stop_words:
                if words[indx] not in dic:
                    dic[words[indx]] = 1
                else:
                    dic[words[indx]] += 1
    return dic

In [14]:
def freq_stop(texts):
    dic = {}
    stop = ['&lt;', '«', '–', '-', '(', ')', '»', 'lt', '', 'gt']

    for text in texts:
        words = text.split()
        for indx, word in enumerate(words):
            word = word.lower()
            words[indx] = word.strip('.,!@?#$%^&*():"№;%=-,')
            if words[indx] not in stop:
                if words[indx] not in dic:
                    dic[words[indx]] = 1
                else:
                    dic[words[indx]] += 1
    return dic

In [41]:
f = open('stats/all_gramm.txt', 'w', encoding='utf-8')
f.write('all texts, without stop words, grammatical forms\n')

49

In [42]:
d = freq(texts) 
i = 0
for w in sorted(d, key=d.get, reverse=True):
    if i < 30:
        f.write(w + ' ' + str(d[w]) + '\n')
    else:
        break
    i += 1

In [43]:
f.close()

In [45]:
f = open('stats/all_stop_gramm.txt', 'w', encoding='utf-8')
f.write('all texts, with stop words, grammatical forms\n')

d = freq_stop(texts) 
i = 0
for w in sorted(d, key=d.get, reverse=True):
    if i < 100:
        f.write(w + ' ' + str(d[w]) + '\n')
    else:
        break
    i += 1
f.close()

In [4]:
thisdir = os.getcwd()  # текущая директория
mystem = os.path.join(thisdir, 'mystem')  # путь к mystem
textdir = os.path.join(thisdir, 'texts')  # путь к нелемматизированным текстам
lemmadir = os.path.join(thisdir, 'lemma_texts')  # путь к лемматизированным текстам
if not os.path.exists(lemmadir):
    os.mkdir(lemmadir)

In [47]:
for d, dirs, files in os.walk(textdir):
    for filename in files:
        print('%s processing...' % filename)
        os.system(mystem + ' -ndl ' + os.path.join(d, filename) + ' ' + os.path.join(lemmadir, filename))
print('Done.')

2065.txt processing...
2685.txt processing...
2682.txt processing...
2846.txt processing...
2658.txt processing...
1673.txt processing...
1794.txt processing...
401.txt processing...
3161.txt processing...
1906.txt processing...
797.txt processing...
2327.txt processing...
836.txt processing...
1646.txt processing...
2999.txt processing...
375.txt processing...
1369.txt processing...
47.txt processing...
1154.txt processing...
2757.txt processing...
450.txt processing...
751.txt processing...
2228.txt processing...
367.txt processing...
1202.txt processing...
3272.txt processing...
3066.txt processing...
1505.txt processing...
2678.txt processing...
1171.txt processing...
1808.txt processing...
1927.txt processing...
1789.txt processing...
3237.txt processing...
3020.txt processing...
648.txt processing...
650.txt processing...
1259.txt processing...
925.txt processing...
1157.txt processing...
1918.txt processing...
97.txt processing...
2969.txt processing...
152.txt processing...
175

235.txt processing...
3213.txt processing...
1075.txt processing...
314.txt processing...
1102.txt processing...
2951.txt processing...
3253.txt processing...
879.txt processing...
1664.txt processing...
2691.txt processing...
49.txt processing...
234.txt processing...
221.txt processing...
432.txt processing...
110.txt processing...
1508.txt processing...
2831.txt processing...
1856.txt processing...
810.txt processing...
2070.txt processing...
2886.txt processing...
1354.txt processing...
274.txt processing...
493.txt processing...
2427.txt processing...
2192.txt processing...
2986.txt processing...
1174.txt processing...
599.txt processing...
3039.txt processing...
422.txt processing...
2312.txt processing...
1545.txt processing...
2180.txt processing...
529.txt processing...
2119.txt processing...
1590.txt processing...
122.txt processing...
1435.txt processing...
1280.txt processing...
2258.txt processing...
2213.txt processing...
1967.txt processing...
343.txt processing...
1599.

3168.txt processing...
2046.txt processing...
2802.txt processing...
686.txt processing...
2558.txt processing...
929.txt processing...
536.txt processing...
1546.txt processing...
369.txt processing...
2877.txt processing...
326.txt processing...
3004.txt processing...
3030.txt processing...
2206.txt processing...
2413.txt processing...
90.txt processing...
209.txt processing...
444.txt processing...
1108.txt processing...
268.txt processing...
2950.txt processing...
1439.txt processing...
1654.txt processing...
1705.txt processing...
1759.txt processing...
1584.txt processing...
317.txt processing...
3214.txt processing...
499.txt processing...
1496.txt processing...
1149.txt processing...
2482.txt processing...
3257.txt processing...
257.txt processing...
1366.txt processing...
2346.txt processing...
575.txt processing...
1979.txt processing...
2447.txt processing...
1930.txt processing...
2919.txt processing...
1802.txt processing...
978.txt processing...
2872.txt processing...
213

380.txt processing...
1217.txt processing...
171.txt processing...
3248.txt processing...
904.txt processing...
1917.txt processing...
399.txt processing...
530.txt processing...
2624.txt processing...
242.txt processing...
2784.txt processing...
2645.txt processing...
2720.txt processing...
2314.txt processing...
443.txt processing...
2422.txt processing...
782.txt processing...
336.txt processing...
3018.txt processing...
429.txt processing...
38.txt processing...
1068.txt processing...
2615.txt processing...
833.txt processing...
1735.txt processing...
1476.txt processing...
713.txt processing...
2117.txt processing...
977.txt processing...
727.txt processing...
1847.txt processing...
400.txt processing...
724.txt processing...
3073.txt processing...
1191.txt processing...
262.txt processing...
2341.txt processing...
2372.txt processing...
577.txt processing...
2329.txt processing...
2045.txt processing...
2759.txt processing...
513.txt processing...
701.txt processing...
246.txt pr

423.txt processing...
2202.txt processing...
1558.txt processing...
2449.txt processing...
809.txt processing...
1437.txt processing...
1521.txt processing...
482.txt processing...
1926.txt processing...
1693.txt processing...
1425.txt processing...
2800.txt processing...
1835.txt processing...
2709.txt processing...
2675.txt processing...
1406.txt processing...
1028.txt processing...
2961.txt processing...
3265.txt processing...
984.txt processing...
655.txt processing...
1410.txt processing...
2338.txt processing...
762.txt processing...
2080.txt processing...
2075.txt processing...
469.txt processing...
1757.txt processing...
1238.txt processing...
1240.txt processing...
1761.txt processing...
2012.txt processing...
1288.txt processing...
526.txt processing...
414.txt processing...
233.txt processing...
2085.txt processing...
149.txt processing...
772.txt processing...
2733.txt processing...
3149.txt processing...
78.txt processing...
162.txt processing...
2579.txt processing...
127

1995.txt processing...
2323.txt processing...
3131.txt processing...
2523.txt processing...
3112.txt processing...
1004.txt processing...
570.txt processing...
211.txt processing...
2433.txt processing...
3155.txt processing...
2328.txt processing...
1677.txt processing...
474.txt processing...
794.txt processing...
156.txt processing...
2465.txt processing...
2899.txt processing...
2225.txt processing...
2462.txt processing...
112.txt processing...
294.txt processing...
1034.txt processing...
3011.txt processing...
634.txt processing...
2104.txt processing...
2099.txt processing...
1719.txt processing...
283.txt processing...
368.txt processing...
255.txt processing...
437.txt processing...
2274.txt processing...
485.txt processing...
2140.txt processing...
1509.txt processing...
4.txt processing...
2836.txt processing...
510.txt processing...
606.txt processing...
2621.txt processing...
2043.txt processing...
616.txt processing...
1143.txt processing...
2741.txt processing...
1833.tx

67.txt processing...
1749.txt processing...
55.txt processing...
1801.txt processing...
1342.txt processing...
229.txt processing...
732.txt processing...
554.txt processing...
1488.txt processing...
1321.txt processing...
6301.txt processing...
68.txt processing...
2798.txt processing...
2601.txt processing...
768.txt processing...
40.txt processing...
464.txt processing...
225.txt processing...
1559.txt processing...
1791.txt processing...
966.txt processing...
1662.txt processing...
1905.txt processing...
1463.txt processing...
1045.txt processing...
1279.txt processing...
1185.txt processing...
2663.txt processing...
218.txt processing...
2236.txt processing...
1222.txt processing...
2444.txt processing...
446.txt processing...
2718.txt processing...
1812.txt processing...
2892.txt processing...
1829.txt processing...
2187.txt processing...
2241.txt processing...
2897.txt processing...
1113.txt processing...
652.txt processing...
267.txt processing...
1118.txt processing...
1363.tx

1564.txt processing...
1767.txt processing...
644.txt processing...
619.txt processing...
28.txt processing...
2633.txt processing...
870.txt processing...
899.txt processing...
2388.txt processing...
1393.txt processing...
1971.txt processing...
1563.txt processing...
2756.txt processing...
1750.txt processing...
1492.txt processing...
333.txt processing...
117.txt processing...
2889.txt processing...
1684.txt processing...
2280.txt processing...
3096.txt processing...
2300.txt processing...
1049.txt processing...
1.txt processing...
1422.txt processing...
296.txt processing...
949.txt processing...
363.txt processing...
1817.txt processing...
965.txt processing...
2298.txt processing...
3009.txt processing...
1114.txt processing...
1640.txt processing...
271.txt processing...
1374.txt processing...
1311.txt processing...
2888.txt processing...
1127.txt processing...
3037.txt processing...
601.txt processing...
132.txt processing...
230.txt processing...
66.txt processing...
2553.txt 

2521.txt processing...
160.txt processing...
2195.txt processing...
709.txt processing...
643.txt processing...
Done.


In [49]:
lemma_texts = []

for d, dirs, files in os.walk(lemmadir):
    for filename in files:
        lemma_texts.append(open('lemma_texts/' + filename, 'r', encoding='utf-8').read())        

In [54]:
f = open('stats/all_lemma.txt', 'w', encoding='utf-8')
f.write('all texts, without stop words, lemmas\n')

d = freq(lemma_texts) 
i = 0
for w in sorted(d, key=d.get, reverse=True):
    if i < 100:
        f.write(w + ' ' + str(d[w]) + '\n')
    else:
        break
    i += 1
f.close()

In [55]:
f = open('stats/all_stop_lemma.txt', 'w', encoding='utf-8')
f.write('all texts, with stop words, lemmas\n')

d = freq_stop(lemma_texts) 
i = 0
for w in sorted(d, key=d.get, reverse=True):
    if i < 100:
        f.write(w + ' ' + str(d[w]) + '\n')
    else:
        break
    i += 1
f.close()

In [73]:
def freq(fname, meta, texts):
    dic = {}

    for text in texts:
        words = text.split()
        for indx, word in enumerate(words):
            word = word.lower()
            words[indx] = word.strip('.,!@?#$%^&*():"№;%=-,')
            if words[indx] not in stop_words:
                if words[indx] not in dic:
                    dic[words[indx]] = 1
                else:
                    dic[words[indx]] += 1
                    
    f = open('stats/' + fname, 'w', encoding='utf-8')
    f.write(meta + '\n')

    i = 0
    for w in sorted(dic, key=dic.get, reverse=True):
        if i < 100:
            f.write(w + ' ' + str(dic[w]) + '\n')
        else:
            break
        i += 1
        
    f.close()

In [18]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="HL"'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

hl_texts = []
for n in rulec:
    req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
    text = ''
    for i in db.execute(req2):
        text += i[0]
#         print(i)
    hl_texts.append(text)

(' Загрязнение тяжелыми металлами Дальнегорского района. ',)
('Одной из самых главных экологических проблем на территории Российской Федерации является загрязнение окружющей среды тяжелыми металлами, такими как свинeц, кадмий и цинк. ',)
('Эта проблема особеннo характерна для тех местностей, где добывается руда и выплавляется свинец - Дальнегорский район Приморского края, долина реки Рудной, пос. ',)
('Рудная Пристань. ',)
('Согласно проведенным исследованиям Тихоокеанского института географии, по уровню загрязнения почв пос. ',)
('Рудная Пристань является одним из пяти самых загрязненных населенных пунктов России. ',)
('В ходе исследования было выявлено высокое содержание тяжелых металлов в воде, атмосферных осадках, почвах и растениях, что оказывает негативное влияние на флору и фауну. ',)
('Уровень загрязнения почвы в промышленным зонах Дальнегорского района достигал 9500 кг при том, что предельно допустимая норма составляет 32 мг свинца на 1 кг почвы. ',)
('Эти данные также указыва

('В чем-то они совпадали, а в чем-то нет, но эта дискуссия позволила нам узнать много нового и интересного о Советском Союзе и современной России и людях, которые жили тогда и сейчас. ',)
(' Mузей - это машина времени. ',)
('Безусловно не в коем случае нельзя спорить, что музей - это машина времени. ',)
('Прежде всего, давайте посмотрим на смысл музеев, зачем нам они? ',)
('Музеи созданы, чтобы рассказать и показать людям будущего о прошлом. ',)
('Во-вторых-это, чтобы мы знали историю. ',)
('В-третьих, чтобы мы учились и брали пример, как поступать и развиваться правильно смотря на поступки и ошибки наших человеческих предков. ',)
('Например, когда мы приходим в военный музей мы переступаем порог времени и уходим во время войны, здесь мы можем смотреть на страшные оружие и фотографии и письма от людей, которые прожили через ужасное время здесь мы можем видеть, какой ущерб война может принести людям и в себе мы можем принять решение, что мы не будем поддерживать войны. ',)
('В заключени

('Она не только образованный человек но она так же очень трудится даже в старости. ',)
('Так же, она является вдовой благородного учёного. ',)
('Виктория Дмитриевна считает, что каждая женщина имеет долг: родить и вырастить детей, а всё остальное - « мелочи жизни». ',)
('Имея традиционую точку зрения, Виктория Дмитриевна является покровителем своего взрослого сына и внука. ',)
('Виктория Дмитриевна - это человек праздник и прекрасный образ женщины, которая многое достигла в не простой стране. ',)
('Проходя стажировку в Санкт-Петербуржской международной бизнес ассоциации ( СПИБА) я работаю в женском коллективе. ',)
('Наблюдая за ними я поняла, что нужно всегда приспосабливаться и меняться, чтобы чего-то достичь в Петербурге. ',)
('Высшее образование это не достаточно и даже не всегда так ценится, как на например, жизненный опыт, а ещё лучше - связи. ',)
('Самой младшей из моих коллег, Тане, двадцать три года. ',)
('Она мечтает о яркой жизни за рубежом, где она может работать в каком не 

('Никто бы не смотрал с презрением на человека инвалида, или некрасивую девочку. ',)
('У всех бы была работа и квартира, и полноценное счатье. ',)
('Сейчас, мы как раз живём в антиутопии. ',)
('В любую минуту может упасть самолёт или ядерная бомба. ',)
('Бедные дети в Африке голодают- а мы строим небоскрёбы. ',)
('Пришёл тот час, о котором с ужасом пишут книги- технология и прогресс- и мир катится вниз. ',)
('Началось глобальное отепление, и мы даже н езнаем сколько лет нам осталось жить на этой планете. ',)
('Утопия и антиутопия- это как небо и ад, и кто выберет ад? ',)
('Утопия, сколько бы не хотелось, пока невозможна. ',)
('Надо сначала изменить отношение людей друг к другу, злобу, ненависть. ',)
('Надо чтобы все стремились к лучшему, и хотели бы жить в спокойном и красивом мире. ',)
('Может быть после тьмы настанет заря, и может мир в котором мы сейчас живём- это предостережение. ',)
(' Экономический факультет в ПГУ. ',)
('Это мой второй год в университете, и я изучаю экономику. ',

('Каждое государствое имеет так называемый локальный код - отношение с окружающими странами- государствами. ',)
('Существует Американская геополитика. ',)
('Америкаская политика изучает концепцию американизма, роль Америки в мировом масштабе, влияние Америки на другие страны. ',)
('Американская геополитика утверждает, что Америка является лидером западного мира. ',)
('Одна из самых важных проблем геополитики это соотношение геополитики и географии. ',)
('Классики геополитики считают значение геополитики результатом синтеза географии и политологии. ',)
('Рудольф Челлен, шведский социолог и политолог, иобрел термин « Геополитика» и описал геополитику как науку о государстве как географическом организме в прострастве. ',)
('Так как Челлен был политологом он считал что эконополитика, демолитика, социополитика, кратополитика, и геополитика взаимосвязаны мезду собой. ',)
('Многие ученые стали рассуждать о научном значении термина геополитика, особенно когда геополитика стала изпользоваться к

...

('Кольцо с жабой валяется на полу. ',)
('Петя добежал до озера и лег под дерево, полежал и побрел домой. ',)
('Тема рассказа детство и взросление, параллельно идущие, с темой фантазией и реальностью. ',)
('Рассказ ведется от третьго лица, но часто создается впечатление, что от лица Пети, потому что передается его отношение к окружающим его людям, событиям и т. ',)
('п. ',)
('У читателя складывается мнение о местах, людях, благодаря Петиному описанию. ',)
('Рассказ называется « Свидание с птицей», с птицей Сирин. ',)
('В конце рассказа дедушка Пети умирает, и Петя « встречается с птицей», что меняет его жизнь, его детство заканчивается. ',)
('Мне очень понравился рассказ. ',)
('Я бы посоветовала всем его прочитать, мне понравилось как Татьяна Толстая описывает такие простые вещи, как таящее масло в каше, как таящий остров в Саргассовом море. ',)
('Рассказы ее легкие для чтения и захватывающие. ',)
(' &lt; Объясните законопроекты 66 и 67&gt;. Здравствуй, Я очень рад что ты наконец то нап

('Универсальные права человека уже всем известны, многими востребованы, и все мы стремимся к светлому будущему, к достойной жизни. ',)
('Но разница между гражданами той или иной страны существует, в том числи и между американской и русской молодежи. ',)
('Эта разница не на поверхности, а заключается глубоко в манере поведения, менталитете, мировоззрение. ',)
('Чтобы нам, молодым людям из США, заметить различия между людьми нашего поколения, проживающими в США и России, необходима много общаться с русскими. ',)
('На данный момент характеристики, которые делают нас разными, не заметны, но со временем они станут более ясными. ',)
('В качестве примера приведу один момент. ',)
('На мой взгляд, русские более самостоятельные и сообразительные. ',)
('Стоит всего попытаться найти спецкурс в российском вузе, чтобы это понять. ',)
('Более того, русская молодежь живет в, так называемой, культуре высокого контекста. ',)
('То есть русский может понять какую-либо ситуацию без прямого объяснения того,

('В тоже время, через политическую можно любого зделать известным, потому что важно не ваше лицо, а лицо на банктноте. ',)
(' &lt; Write an advertisement for a caf&amp;#233;&gt;. Вы хотите открыть кафе, но не можете найти помещение для него? ',)
('Мы рады Вам объявить, что появилась возможность купить замечательное помещение, которое находится в центре города Портленд на улице Тэйлор! ',)
('Это прекрасное помещение довольно новое и содержит зал, кухню и туалет. ',)
('Оно идеальное для открытие небольшого и уютного кафе. ',)
('Не упустите Ваш шанс, у покупайте это помещение! ',)
(' Термин – это слово, которое имеет конкретное значение. ',)
('Юриспруденция – это правоведение, совокупность наук о правде; практическая деятельность юристов, решения судебных органов. ',)
('Термины нужны, чтобы дать слову специфическое значение. ',)
('Термины используются, чтобы выявить конкретное определение какого-либо слова. ',)
('Профессиональный жаргон – это жаргон, который используется людьми, которые п

('Другие страны захватывают водные ресурсы рек строением плотин, тем самым влияя на политическую и экономическую ситуацию в странах, расположенных вниз по течению. ',)
('Наличие водного кризиса подтверждается и статистическими данными и прогнозами. ',)
('Семьдесят два процента всей используемой государством пресной воды уходит на орошения, а к 2030 году эта потребность увеличится. ',)
('В насоящее время одна пятая людей не имеет доступа к чистой питьевой воде. ',)
('Только менее одного процента пресной воды доступна для использования нами. ',)
('.На юге Африки около трёхсот сорока миллионов человек не имеют доступа к безопасной питьевой воде. ',)
('Смерть трёх миллионов человек каждый год связана с качеством воды. ',)
('Каждые семнадцать секунд умерает по ребёнку. ',)
('Итак, лишь малая часть воды пригодна для питья. ',)
('Она постоянно загрязняется. ',)
('Потребность на пресную воду растёт. ',)
('Множество людей умирает от жажды. ',)
('Возникают конфликты между странами из- за водных 

('На мой взгляд, от вышеперечисленных причин, все же чаще всего погибают именно мужчины. ',)
('Соответственно данным криминальной карты России наркомания стоит на втором месте по всей стране. ',)
(' (&lt; link&gt; здесь также можно посмотреть на отдельные регионы). ',)
('По словам директора Федеральной службы РФ по наркоконтролю Виктора Иванова, ежегодно в России умирает 100 тысяч от наркотиков. ',)
('Вот вам, собственно говоря, одна из причин почему смертность мужчин в России выше смертности женщин. ',)
('Если кому- то интересно, по ссылке можно посмотреть на статистику соотношения женщин и мужчин в разных странах и сравнить их: &lt; link&gt;. Библиография. ',)
(' &lt; Что значит для вас понятие граница?&gt;. Я считаю, что для каждго человека существует несколько понятий слоава граница и у каждого они немного отличаются. ',)
('Для меня это слово имеет три понятия. ',)
('Первое это прямое понятие или словарное. ',)
('С этим понятием я часто встречаюсь на уроках истории или географии, и

In [69]:
len(hl_texts)

1275

In [70]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="FL"'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

fl_texts = []
for n in rulec:
    req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
    text = ''
    for i in db.execute(req2):
        text += i[0]
    fl_texts.append(text)

In [71]:
len(fl_texts)

1449

In [79]:
freq('hl_gram.txt', 'heritage, without stop words, grammatical forms', hl_texts)

In [76]:
freq('fl_gram.txt', 'foreign, without stop words, grammatical forms', fl_texts)

In [78]:
freq_stop('hl_stop_gram.txt', 'heritage, with stop words, grammatical forms', hl_texts)

In [80]:
freq_stop('fl_stop_gram.txt', 'foreign, with stop words, grammatical forms', fl_texts)

In [89]:
def lemma(texts):
    thisdir = os.getcwd()  # текущая директория
    mystem = os.path.join(thisdir, 'mystem')  # путь к mystem
    textdir = os.path.join(thisdir, 'texts_temp')  # путь к нелемматизированным текстам
    lemmadir = os.path.join(thisdir, 'lemma_temp')  # путь к лемматизированным текстам
    
    for indx, text in enumerate(texts):
        f = open('texts_temp/' + str(indx) + '.txt', 'w', encoding='utf-8')
        f.write(text)
        f.close()
        
    for d, dirs, files in os.walk(textdir):
        for filename in files:
            os.system(mystem + ' -ndl ' + os.path.join(d, filename) + ' ' + os.path.join(lemmadir, filename))
    print('Done.')
    
    lemma_texts = []

    for d, dirs, files in os.walk(lemmadir):
        for filename in files:
            lemma_texts.append(open('lemma_temp/' + filename, 'r', encoding='utf-8').read()) 
    
    return lemma_texts

In [90]:
fl_lemma = lemma(fl_texts)
hl_lemma = lemma(hl_texts)

Done.
Done.


In [91]:
len(fl_lemma)

1449

In [42]:
def freq_stop(fname, meta, texts):
    dic = {}
    number = 0 # сколько всего слов
    
    for text in texts:
        words = text.split()
        for indx, word in enumerate(words):
            number += 1
            word = word.lower()
            words[indx] = word.strip('.,!@?#$%^&*():"№;%=-,')
            if words[indx] not in stop_words:
                if words[indx] not in dic:
                    dic[words[indx]] = 1
                else:
                    dic[words[indx]] += 1
                    
    f = open('../stats/' + fname, 'w', encoding='utf-8')
    f.write(meta + '\n')

    i = 0
    for w in sorted(dic, key=dic.get, reverse=True):
        if i < 100:
            f.write(w + ' ' + str(dic[w]) + ' ' + str(round(dic[w] / number * 1000000, 2)) + '\n')
        else:
            break
        i += 1
    print(number)
        
    f.close()

In [5]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="FL" AND date1=2009'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

fl_start_texts = []
for n in rulec:
    f = open('fl/start/' + str(n) + '.txt', 'w', encoding='utf-8')
    req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
    text = ''
    for i in db.execute(req2):
        text += i[0]
    fl_start_texts.append(text)
    f.write(text)
    f.close()

In [6]:
len(fl_start_texts)

181

In [7]:
for d, dirs, files in os.walk('fl/start/'):
    for filename in files:
        print('%s processing...' % filename)
        os.system(mystem + ' -ndl ' + os.path.join(d, filename) + ' ' + os.path.join('fl/start_lemma', filename))
print('Done.')

1147.txt processing...
1107.txt processing...
1187.txt processing...
779.txt processing...
2364.txt processing...
632.txt processing...
525.txt processing...
1506.txt processing...
396.txt processing...
1969.txt processing...
3170.txt processing...
2570.txt processing...
1042.txt processing...
1838.txt processing...
434.txt processing...
26.txt processing...
3213.txt processing...
3253.txt processing...
432.txt processing...
110.txt processing...
584.txt processing...
1051.txt processing...
236.txt processing...
1403.txt processing...
312.txt processing...
663.txt processing...
1383.txt processing...
231.txt processing...
2176.txt processing...
873.txt processing...
1530.txt processing...
1511.txt processing...
1546.txt processing...
3030.txt processing...
1759.txt processing...
1930.txt processing...
546.txt processing...
1786.txt processing...
947.txt processing...
1655.txt processing...
3005.txt processing...
2830.txt processing...
1179.txt processing...
2432.txt processing...
2960.

In [16]:
lemma_texts_fl_start = []

for d, dirs, files in os.walk('../fl/start_lemma/'):
    for filename in files:
        lemma_texts_fl_start.append(open('../fl/start_lemma/' + filename, 'r', encoding='utf-8').read())        

In [43]:
freq_stop('fl_start_lemma.txt', 'L2 speakers, year 2009, frequent lemmas', lemma_texts_fl_start)

32449


In [18]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="FL" AND date1=2012'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

fl_end_texts = []
for n in rulec:
    f = open('fl/end/' + str(n) + '.txt', 'w', encoding='utf-8')
    req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
    text = ''
    for i in db.execute(req2):
        text += i[0]
    fl_end_texts.append(text)
    f.write(text)
    f.close()

In [19]:
len(fl_end_texts)

525

In [20]:
for d, dirs, files in os.walk('fl/end/'):
    for filename in files:
        print('%s processing...' % filename)
        os.system(mystem + ' -ndl ' + os.path.join(d, filename) + ' ' + os.path.join('fl/end_lemma', filename))
print('Done.')

2658.txt processing...
1906.txt processing...
2757.txt processing...
367.txt processing...
1505.txt processing...
1157.txt processing...
1753.txt processing...
2018.txt processing...
83.txt processing...
2573.txt processing...
1855.txt processing...
2966.txt processing...
1539.txt processing...
2839.txt processing...
2652.txt processing...
1796.txt processing...
2884.txt processing...
1960.txt processing...
3045.txt processing...
377.txt processing...
104.txt processing...
2318.txt processing...
2569.txt processing...
730.txt processing...
1084.txt processing...
778.txt processing...
1770.txt processing...
2313.txt processing...
2204.txt processing...
2925.txt processing...
1001.txt processing...
738.txt processing...
880.txt processing...
783.txt processing...
1308.txt processing...
1779.txt processing...
699.txt processing...
1037.txt processing...
1589.txt processing...
2175.txt processing...
2376.txt processing...
2415.txt processing...
1582.txt processing...
2477.txt processing...

2679.txt processing...
1849.txt processing...
2056.txt processing...
2637.txt processing...
2337.txt processing...
3075.txt processing...
1074.txt processing...
2697.txt processing...
773.txt processing...
520.txt processing...
895.txt processing...
1611.txt processing...
3255.txt processing...
2976.txt processing...
179.txt processing...
1854.txt processing...
1257.txt processing...
3218.txt processing...
1744.txt processing...
95.txt processing...
2221.txt processing...
2064.txt processing...
2953.txt processing...
3127.txt processing...
3027.txt processing...
67.txt processing...
1749.txt processing...
1342.txt processing...
229.txt processing...
732.txt processing...
6301.txt processing...
68.txt processing...
40.txt processing...
966.txt processing...
1463.txt processing...
2663.txt processing...
2241.txt processing...
1118.txt processing...
1363.txt processing...
1912.txt processing...
74.txt processing...
1657.txt processing...
185.txt processing...
912.txt processing...
2734.tx

In [15]:
lemma_texts_fl_end = []

for d, dirs, files in os.walk('../fl/end_lemma/'):
    for filename in files:
        lemma_texts_fl_end.append(open('../fl/end_lemma/' + filename, 'r', encoding='utf-8').read())        

In [6]:
len(lemma_texts_fl_end)

525

In [44]:
freq_stop('fl_end_lemma.txt', 'L2 speakers, year 2012, frequent lemmas', lemma_texts_fl_end)

97187


In [37]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="HL" AND date1=2009'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

hl_start_texts = []
for n in rulec:
    f = open('hl/start/' + str(n) + '.txt', 'w', encoding='utf-8')
    req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
    text = ''
    for i in db.execute(req2):
        text += i[0]
    hl_start_texts.append(text)
    f.write(text)
    f.close()

In [38]:
len(hl_start_texts)

474

In [39]:
for d, dirs, files in os.walk('hl/start/'):
    for filename in files:
        print('%s processing...' % filename)
        os.system(mystem + ' -ndl ' + os.path.join(d, filename) + ' ' + os.path.join('hl/start_lemma', filename))
print('Done.')

2846.txt processing...
1794.txt processing...
401.txt processing...
3161.txt processing...
1369.txt processing...
450.txt processing...
1808.txt processing...
925.txt processing...
97.txt processing...
216.txt processing...
139.txt processing...
848.txt processing...
3241.txt processing...
1387.txt processing...
2817.txt processing...
1243.txt processing...
1692.txt processing...
1617.txt processing...
1237.txt processing...
1888.txt processing...
2416.txt processing...
1695.txt processing...
131.txt processing...
2752.txt processing...
636.txt processing...
1473.txt processing...
1601.txt processing...
556.txt processing...
874.txt processing...
2629.txt processing...
2820.txt processing...
1624.txt processing...
3169.txt processing...
535.txt processing...
1551.txt processing...
45.txt processing...
3210.txt processing...
3224.txt processing...
2635.txt processing...
1419.txt processing...
424.txt processing...
3120.txt processing...
917.txt processing...
1586.txt processing...
408.t

3147.txt processing...
1046.txt processing...
3271.txt processing...
2879.txt processing...
1029.txt processing...
1272.txt processing...
533.txt processing...
3103.txt processing...
735.txt processing...
1782.txt processing...
497.txt processing...
2739.txt processing...
2256.txt processing...
3082.txt processing...
2848.txt processing...
695.txt processing...
1303.txt processing...
1066.txt processing...
2151.txt processing...
2760.txt processing...
2435.txt processing...
226.txt processing...
1145.txt processing...
1697.txt processing...
1452.txt processing...
1803.txt processing...
3118.txt processing...
566.txt processing...
903.txt processing...
279.txt processing...
304.txt processing...
1085.txt processing...
1484.txt processing...
25.txt processing...
1378.txt processing...
1890.txt processing...
907.txt processing...
1831.txt processing...
2844.txt processing...
126.txt processing...
2504.txt processing...
50.txt processing...
105.txt processing...
1823.txt processing...
205.

In [14]:
lemma_texts_hl_start = []

for d, dirs, files in os.walk('../hl/start_lemma/'):
    for filename in files:
        lemma_texts_hl_start.append(open('../hl/start_lemma/' + filename, 'r', encoding='utf-8').read())        

In [8]:
len(lemma_texts_hl_start)

474

In [45]:
freq_stop('hl_start_lemma.txt', 'Heritage speakers, year 2009, frequent lemmas', lemma_texts_hl_start)

108016


In [49]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="HL" AND date1=2012'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

hl_end_texts = []
for n in rulec:
    f = open('hl/end/' + str(n) + '.txt', 'w', encoding='utf-8')
    req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
    text = ''
    for i in db.execute(req2):
        text += i[0]
    hl_end_texts.append(text)
    f.write(text)
    f.close()

In [50]:
len(hl_end_texts)

246

In [51]:
for d, dirs, files in os.walk('hl/end/'):
    for filename in files:
        print('%s processing...' % filename)
        os.system(mystem + ' -ndl ' + os.path.join(d, filename) + ' ' + os.path.join('hl/end_lemma', filename))
print('Done.')

836.txt processing...
3020.txt processing...
1259.txt processing...
2222.txt processing...
1397.txt processing...
819.txt processing...
594.txt processing...
2428.txt processing...
3097.txt processing...
715.txt processing...
817.txt processing...
2851.txt processing...
680.txt processing...
763.txt processing...
2406.txt processing...
1570.txt processing...
1148.txt processing...
2973.txt processing...
1128.txt processing...
1721.txt processing...
1281.txt processing...
2361.txt processing...
894.txt processing...
923.txt processing...
2255.txt processing...
1260.txt processing...
2951.txt processing...
2831.txt processing...
493.txt processing...
529.txt processing...
1967.txt processing...
1599.txt processing...
136.txt processing...
538.txt processing...
1478.txt processing...
2165.txt processing...
2536.txt processing...
506.txt processing...
1194.txt processing...
1225.txt processing...
2265.txt processing...
394.txt processing...
716.txt processing...
1443.txt processing...
2514

In [12]:
lemma_texts_hl_end = []

for d, dirs, files in os.walk('../hl/end_lemma/'):
    for filename in files:
        lemma_texts_hl_end.append(open('../hl/end_lemma/' + filename, 'r', encoding='utf-8').read())        

In [13]:
len(lemma_texts_hl_end)

246

In [46]:
freq_stop('hl_end_lemma.txt', 'Heritage speakers, year 2012, frequent lemmas', lemma_texts_hl_end)

40539


# Collocations

In [13]:
import nltk

In [11]:
def get_collocations(fname, meta, texts):
    lemmed = []
    
    for text in texts:
        words = text.split()
        for indx, word in enumerate(words):
            word = word.lower()
            words[indx] = word.strip('.,!@?#$%^&*():"№;%=-,')
            if words[indx] not in stop_words:
                lemmed.append(words[indx])
                    
    f = open('stats/' + fname, 'w', encoding='utf-8')
    f.write(meta + '\n')
    
    bg = list(nltk.bigrams(lemmed))
    bgfd = nltk.FreqDist(bg)
    collocats = bgfd.most_common()
    
    for c in collocats:
        f.write(c[0][0] + ' ' + c[0][1] + ' ' + str(c[1]) + '\n')

    f.close()

In [14]:
get_collocations('fl_start_colloc.txt', 'L2 speakers, year 2009, frequent collocations', lemma_texts_fl_start)

In [15]:
get_collocations('fl_end_colloc.txt', 'L2 speakers, year 2012, frequent collocations', lemma_texts_fl_end)

In [16]:
get_collocations('hl_start_colloc.txt', 'Heritage speakers, year 2009, frequent collocations', lemma_texts_hl_start)

In [17]:
get_collocations('hl_end_colloc.txt', 'Heritage speakers, year 2012, frequent collocations', lemma_texts_hl_end)

# Lexical diversity

In [3]:
from lexicalrichness import LexicalRichness

In [8]:
def lexical_diversity(texts):

    lemmed = []

    for text in texts:
        words = text.split()
        lemmed += words
    
    lex = LexicalRichness(' '.join(lemmed))
        
    print('TTR: ', str(lex.ttr))
    print('MTLD: ', str(lex.mtld(threshold=0.72)))

In [10]:
lexical_diversity(lemma_texts_fl_start)

TTR:  0.11078582434514637
MTLD:  89.4806853452


In [12]:
lexical_diversity(lemma_texts_fl_end)

TTR:  0.07673786887270033
MTLD:  97.0630339235


In [14]:
lexical_diversity(lemma_texts_hl_start)

TTR:  0.06745172100946138
MTLD:  116.552642177


In [9]:
lexical_diversity(lemma_texts_hl_end)

TTR:  0.12220331039246159
MTLD:  107.024129032


In [37]:
def freq_start_end(start, end, fname, meta):
    dic_start = {}
    dic_end = {}
    number = 0 # сколько всего слов
    
    for text in start:
        words = text.split()
        for indx, word in enumerate(words):
            number += 1
            word = word.lower()
            words[indx] = word.strip('.,!@?#$%^&*():"№;%=-,')
            if words[indx] not in stop_words:
                if words[indx] not in dic_start:
                    dic_start[words[indx]] = 1
                else:
                    dic_start[words[indx]] += 1
                    
                    
    for text in end:
        words = text.split()
        for indx, word in enumerate(words):
            number += 1
            word = word.lower()
            words[indx] = word.strip('.,!@?#$%^&*():"№;%=-,')
            if words[indx] not in stop_words:
                if words[indx] not in dic_end:
                    dic_end[words[indx]] = 1
                else:
                    dic_end[words[indx]] += 1
                    
    f = open('../stats/' + fname + '.csv', 'w', encoding='utf-8')
    f.write(meta + '\n')
    
    start_freq = sorted(dic_start, key=dic_start.get, reverse=True)[:100]
    end_freq = sorted(dic_end, key=dic_end.get, reverse=True)[:100]
    general = set(start_freq) & set(end_freq)

    
    for w in general:
        f.write(w + ',' + str(dic_start[w]) + ',' + str(dic_end[w]) + ',' + 
                str(dic_start[w] - dic_end[w]) + ',' + str(dic_start[w] + dic_end[w]) + '\n')
        
    f.close()

In [38]:
freq_start_end(lemma_texts_fl_start, lemma_texts_fl_end, 'L2_lemmas', 'Lemma,2009,2012,Разница,Общая')

In [39]:
freq_start_end(lemma_texts_hl_start, lemma_texts_hl_end, 'HL_lemmas', 'Lemma,2009,2012,Разница,Общая')